In [24]:
import pandas as pd
import requests
import json
import numpy as np


In [25]:
filepath = """../2019 Food Access Research Atlas Data/Food Access Research Atlas.csv"""
data = pd.read_csv(filepath)

In [51]:
texas = data.loc[data["State"]=="Texas",:]
texas = texas[['CensusTract','State','County','Urban','Pop2010','OHU2010','LILATracts_1And10','LILATracts_halfAnd10','HUNVFlag','LowIncomeTracts',
'PovertyRate','MedianFamilyIncome','LA1and10','LAhalfand10','LATracts_half','LALOWI1_10','LALOWI05_10','lahunvhalf','lahunvhalfshare']]

In [27]:
# Open McDonald's JSON file
json_file = open("mcdata.json")
mcdonalds = json.load(json_file)["features"]

# Store all the McDonalds in Texas
mcdonalds_tx = [store for store in mcdonalds if store['properties']['state']=='TX']

# Store lats and lons of each McDonald's restaurant in US
lats = [store['geometry']['coordinates'][1] for store in mcdonalds_tx]
lons = [store['geometry']['coordinates'][0] for store in mcdonalds_tx]

In [28]:
# Store the base URL - this API takes geographical coordinates and returns the Census Tract that those coordinates fall into
base_url = "https://geo.fcc.gov/api/census/area?"

# Initialize an empty list to store the census tract info for each store
census_tracts = []

# Loop through the stores in texas using enumerate
for i in range(len(mcdonalds_tx)):

    # create a url for each store using the latitude and longitutde and base
    store_url = f'{base_url}lat={lats[i]}&lon={lons[i]}&censusYear=2010&format=json'

   # Run API request for each store
    try:
        # Store the results
       results = requests.get(store_url).json()
       
       # Store the tract number - NOTE: only the first 11 digits because the API number is more specific than the Food Access Research Atlas
       tract = results["results"][0]["block_fips"][0:-4]

        # Append the tract number to a list
       census_tracts.append(tract)

    except:
       print("No tract available")
       pass


In [121]:
census_tracts = [int(tract) for tract in census_tracts]
mctracts = pd.DataFrame(census_tracts, columns=["CensusTract"])
mctracts["HasMcD"] = 1
mctracts_grp = mctracts.groupby("CensusTract")
mctracts2 = mctracts.groupby("CensusTract")["HasMcD"].sum()
mctracts = mctracts.drop_duplicates("CensusTract")
mctracts["NumMcD"] = mctracts2

/var/folders/qv/k02r_ndj5gs6km5fylp92pwc0000gp/T/ipykernel_77337/407928579.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mctracts["NumMcD"] = mctracts2


In [ ]:
tx_tracts = pd.DataFrame(texas["CensusTract"],columns=["CensusTract"])
test = pd.merge(tx_tracts,mctracts,how="right",on="CensusTract")
test.count()

In [30]:
len(mcdonalds_tx)

1201